# Specific Objectives

Analyze and select appropriate copula model that captures the dependence structure between
equities and foreign exchange effectively.


In [1]:
import pandas as pd
from scipy.stats import kendalltau
from copulas.multivariate import GaussianMultivariate
from copulas.visualization import scatter_2d

### Step 1 - Data processing

- Get the data

In [2]:
data = pd.read_csv("data/NSE_20_Share_Historical_Data.csv")

- Convert Date column to datetime

In [4]:
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2023-12-29,"74,773.85","74,480.29","74,838.76","74,366.97",368.60M,0.36%
1,2023-12-28,"74,502.58","73,768.64","74,502.58","73,734.59",384.58M,0.99%
2,2023-12-27,"73,768.64","74,034.23","74,249.15","73,762.07",432.91M,-0.34%
3,2023-12-22,"74,023.27","74,288.88","74,430.47","74,018.72",423.20M,-0.36%
4,2023-12-21,"74,289.02","73,397.71","74,292.76","73,397.71",760.93M,1.21%


- Remove commas and convert numeric columns to float

In [5]:
numeric_columns = ['Price', 'Open', 'High', 'Low']

for col in numeric_columns:
    data[col] = data[col].str.replace(',', '').astype(float)

- Preprocess 'Vol.' column

In [6]:
def preprocess_volume(volume):
    if isinstance(volume, str):
        if volume.endswith('M'):
            return float(volume[:-1]) * 1e6  # Multiply by 1 million
        elif volume.endswith('B'):
            return float(volume[:-1]) * 1e9  # Multiply by 1 billion
        else:
            return float(volume)
    else:
        return volume

data['Vol.'] = data['Vol.'].apply(preprocess_volume)

- Preprocess the 'Change %' column

In [7]:
data['Change %'] = data['Change %'].str.replace("%", '').astype(float)

Now the data is ready for further analysis

In [8]:
data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2023-12-29,74773.85,74480.29,74838.76,74366.97,368600000.0,0.36
1,2023-12-28,74502.58,73768.64,74502.58,73734.59,384580000.0,0.99
2,2023-12-27,73768.64,74034.23,74249.15,73762.07,432910000.0,-0.34
3,2023-12-22,74023.27,74288.88,74430.47,74018.72,423200000.0,-0.36
4,2023-12-21,74289.02,73397.71,74292.76,73397.71,760930000.0,1.21


### Step 2: Statistical Analysis

Calculate Kendall's tau correlation coefficient between equities and foreign exchange

In [9]:
kendall_corr, _ = kendalltau(data['Price'], data['Change %'])
print("Kendall's tau correlation coefficient:", kendall_corr)

Kendall's tau correlation coefficient: 0.05937019894451334


### Step 3: Copula Model Selection
- Based on the characteristics of the data, select candidate copula models (e.g., Gaussian copula, t-copula, etc.)


### Step 4: Model Fitting and Evaluation
- Fit selected copula models to the data

In [10]:
copula = GaussianMultivariate()
copula.fit(data[['Price', 'Change %']])

C:\tools\Manim\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
C:\tools\Manim\Lib\site-packages\scipy\stats\_continuous_distns.py:719: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\tools\Manim\Lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\tools\Manim\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)


### Step 5: Model Comparison and Selection
- Evaluate the fitted copula model and compare it with other candidate models
- Visualize the dependence structure

In [11]:
scatter_2d(data[['Price', 'Change %']])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Data=Real<br>Price=%{x}<br>Change %=%{y}<extra></extra>',
              'legendgroup': 'Real',
              'marker': {'color': '#000036', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Real',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([74773.85, 74502.58, 73768.64, ..., 38676.12, 38187.28, 38264.79]),
              'xaxis': 'x',
              'y': array([ 0.36,  0.99, -0.34, ...,  1.28, -0.2 ,  0.06]),
              'yaxis': 'y'}],
    'layout': {'font': {'size': 18},
               'legend': {'title': {'text': 'Data'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'plot_bgcolor': '#F5F5F8',
               'showlegend': False,
               'template': '...',
               'title': {'text': "Data for columns 'Price' and 'Change %'"},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Price'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Change %'}}}
})